In [12]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [13]:
### Fixer la version 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [14]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [15]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [16]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [30]:
# Recherche des chemins des inputs et formatage de l'ouput 
json_path="galaxy_inputs/galaxy_inputs.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file=$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')

echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"


output_file="outputs/outputfile.$extension"
format=""


Input file path: galaxy_inputs/wolfdiet_obigrep.fastq
Nom de fichier (sans chemin, sans extension) : wolfdiet_obigrep
Extension détectée : fastq


In [3]:
#### Variables
listarg=(seq def ident attrib hasattrib predicat)

for i in "${listarg[@]}"; do
    if [[ -d "galaxy_inputs/$i" ]]; then
        value=$(head -n 1 "galaxy_inputs/$i"/*.txt)  # lit la 1ère ligne du fichier .txt dans ce dossier
        declare "$i=$value"                          # crée une variable dynamique nommée comme $i
    fi
done


# Test d'affichage
echo "seq: $seq"
echo "def: $def"
echo "ident: $ident"
echo "attrib: $attrib"
echo "hasattrib: $hasattrib"
echo "predicat: $predicat"


# lmax=""
lmax=$(jq -r '.lmax' galaxy_inputs/galaxy_inputs.json)

# lmin=""   
lmin=$(jq -r '.lmin' galaxy_inputs/galaxy_inputs.json)
####

echo "$input_file"
echo "$seq"

seq: 
def: 
ident: 
attrib: 
hasattrib: 
predicat: 
bash: jq: command not found
bash: jq: command not found




In [43]:
### Ajout de la particule de l'argument 
if [[ -n "$seq"  && "$seq" != "null" ]]; then
    seq="-s $seq"
    echo "variable seq présente: $seq"
else 
    seq=""
fi

if [[ -n "$def"  && "$def" != "null"  ]]; then
    def="-D $def"
    echo "variable def présente : $def"
else 
    def=""
fi

if [[ -n "$ident"  && "$ident" != "null" ]]; then
    ident="-I $ident"
    echo "variable ident présente : $ident"
else 
    ident=""
fi

if [[ -n "$idlist" && "$idlist" != "null" ]]; then
    idlist="--id-list $idlist"
    echo "variable idlist présente : $idlist"
else 
    idlist=""
fi

if [[ -n "$attrib" && "$attrib" != "null" ]]; then
    attrib="-A $attrib"
    echo "variable attrib présente : $attrib"
else 
    attrib=""
fi

if [[ -n "$hasattrib"  && "$hasattrib" != "null" ]]; then
    hasattrib="-a $hasattrib"
    echo "variable hassattribute présente : $hasattrib"
else 
    hasattrib=""
fi

if [[ -n "$predicat"  && "$predicat" != "null" ]]; then
    predicat="-p $predicat"
    echo "variable predicat présente : $predicat"
else 
    predicat=""
fi

if [[ -n "$lmax"  && "$lmax" != "null" ]]; then
    lmax="-L $lmax"
    echo "variable lmax présente : $lmax"
else 
    lmax=""
fi

if [[ -n "$lmin"  && "$lmin" != "null" ]]; then
    lmin="-l $lmin"
    echo "variable lmin présente : $lmin"
else 
    lmin=""
fi

variable predicat présente : -p mode!="joined"


In [44]:
./obitools4/build/obigrep $seq $def $ident $idlist $attrib $hasattrib $predicat $lmax $lmin $input_file --out $output_file --no-order

INFO[0000] Number of workers set 125                    
INFO[0000] Found 1 files to process                     
INFO[0000] galaxy_inputs/wolfdiet_obigrep.fastq mime type: text/fastq 
INFO[0000] On output use JSON headers                   
| Reading sequences (41435/-, 9924 it/s) [4s]  
